# Exploring the BBBC021 dataset

In [ ]:
%load_ext autoreload
%autoreload 2

# Imports

In [ ]:
import holoviews as hv
import numpy as np

from pybbbc import BBBC021

# Initializing HoloViews

In [ ]:
hv.extension("bokeh")

In [ ]:
im_opts = hv.opts.Image(
    aspect="equal",
    tools=["hover"],
    active_tools=["wheel_zoom"],
    colorbar=True,
    cmap="fire",
    normalize=False,
)

rgb_opts = hv.opts.RGB(
    aspect="equal",
    active_tools=["wheel_zoom"],
)

hv.opts.defaults(im_opts, rgb_opts)

# BBBC021 class

* What it does for us
* How to view images with HoloViews
* Subset the data - compounds with known MoA and compounds without

In [ ]:
bbbc021 = BBBC021()

In [ ]:
bbbc021.CHANNELS

In [ ]:
from pybbbc.utils import bytes_to_str

In [ ]:
bytes_to_str([b"asdf", b"3134"])

In [ ]:
bbbc021.moa

In [ ]:
bbbc021.image_df.dtypes

In [ ]:
moas = bbbc021.moa_df["moa"].unique()
num_classes = len(moas[moas != "null"])

num_classes

In [ ]:
len(bbbc021.image_df)

In [ ]:
moas = bbbc021.moa_df.sort_values("moa")["moa"].unique()
moas = moas[moas != "null"]

mapping = {idx: moa for idx, moa in enumerate(moas)}
mapping

In [ ]:
moas = bbbc021.moa_df["moa"].unique()

In [ ]:
bbbc021.COMPOUNDS[:5]

In [ ]:
bbbc021.IMG_SHAPE

In [ ]:
bbbc021.N_SITES

In [ ]:
bbbc021.PLATES[:3]

In [ ]:
bbbc021.MOA

In [ ]:
bbbc021.image_df

In [ ]:
bbbc021.moa_df

In [ ]:
bbbc021.CHANNELS

In [ ]:
bbbc021_with_moa = BBBC021(moa=[moa for moa in BBBC021.MOA if moa != "null"])

In [ ]:
bbbc021_with_moa.moa_df["moa"].unique()

In [ ]:
bbbc021_with_moa.moa_df["concentration"].unique().tolist()

In [ ]:
bbbc021.CHANNELS

In [ ]:
def make_layout(image_idx):
    image, metadata = bbbc021[image_idx]

    prefix = f"{metadata.compound.compound} @ {metadata.compound.concentration:.2e} μM, {metadata.compound.moa}"

    plots = []

    cmaps = ["fire", "kg", "kb"]

    for channel_idx, im_channel in enumerate(image):
        plot = hv.Image(
            im_channel,
            bounds=(0, 0, im_channel.shape[1], im_channel.shape[0]),
            label=f"{prefix} | {bbbc021.CHANNELS[channel_idx]}",
        ).opts(cmap=cmaps[channel_idx])
        plots.append(plot)

    plots.append(
        hv.RGB(
            image.transpose(1, 2, 0),
            bounds=(0, 0, im_channel.shape[1], im_channel.shape[0]),
            label="Channel overlay",
        )
    )

    return hv.Layout(plots).cols(2)


hv.DynamicMap(make_layout, kdims="image").redim.range(image=(0, len(bbbc021) - 1))

# Explore by MoA